In [1]:
import pyspark 
from pyspark.sql import SparkSession, DataFrame
from pyspark import SparkConf, SparkContext
import os
import re
import numpy as np
import time
from functools import reduce

Location_master = 'spark://linux:7077'
# Location_master = 'local'
try:
    sparkSession.stop()
except:
    pass
sparkSession = SparkSession \
    .builder \
    .master(Location_master)\
    .appName("Process 2020") \
    .config("spark.some.config.option", "some-value") \
    .config("spark.executor.memory", "4g") \
    .config('spark.cores.max','4')\
    .getOrCreate()

In [2]:
data_dir = '../Diemthi2020/'
data = sparkSession.read.csv(data_dir + '*/', header=True)
data_bosung = sparkSession.read.csv('../Bosung/*/', header=True)

data.show()
data_bosung.show()

+---+--------+--------+----+-------+---------+------+-------+--------+---------+-------+------+----+---------+
|_c0|     SBD| Cum_thi|Toan|Ngu_van|Ngoai_ngu|Vat_ly|Hoa_hoc|Sinh_hoc|Diem_KHTN|Lich_su|Dia_ly|GDCD|Diem_KHXH|
+---+--------+--------+----+-------+---------+------+-------+--------+---------+-------+------+----+---------+
|  0|64002156|64002156|   7|   5.25|      3.4|  7.25|      6|    5.25|     null|   null|  null|null|     null|
|  1|64002157|64002157| 5.4|   6.25|      1.8|  null|   null|    null|     null|      4|  null|   8|     null|
|  2|64002158|64002158| 4.8|      7|        4|  null|   null|    null|     null|   5.25|  null|   8|     null|
|  3|64002159|64002159|   6|   6.75|      3.2|  null|   null|    null|     null|   3.25|  null|8.75|     null|
|  4|64002160|64002160| 5.2|      7|      2.8|  null|   null|    null|     null|    5.5|  null| 8.5|     null|
|  5|64002161|64002161| 6.8|   6.75|        4|  null|   null|    null|     null|      5|  null|   9|     null|
|

In [3]:
data = data.drop('_c0', 'Cum_thi', 'Diem_KHTN', 'Diem_KHXH')\
    .select("SBD", 'Toan', "Ngu_van", "Ngoai_ngu", "Vat_ly", "Hoa_hoc", "Sinh_hoc", "Lich_su", "Dia_ly", "GDCD")

data_bosung = data.drop('_c0', 'Cum_thi', 'Diem_KHTN', 'Diem_KHXH')\
    .select("SBD", 'Toan', "Ngu_van", "Ngoai_ngu", "Vat_ly", "Hoa_hoc", "Sinh_hoc", "Lich_su", "Dia_ly", "GDCD")

data_all = reduce(DataFrame.union, [data, data_bosung])
data_all = data_all.drop_duplicates()
data_all.printSchema()
print(data.count())
data.show(10)

root
 |-- SBD: string (nullable = true)
 |-- Toan: string (nullable = true)
 |-- Ngu_van: string (nullable = true)
 |-- Ngoai_ngu: string (nullable = true)
 |-- Vat_ly: string (nullable = true)
 |-- Hoa_hoc: string (nullable = true)
 |-- Sinh_hoc: string (nullable = true)
 |-- Lich_su: string (nullable = true)
 |-- Dia_ly: string (nullable = true)
 |-- GDCD: string (nullable = true)

834375
+--------+----+-------+---------+------+-------+--------+-------+------+----+
|     SBD|Toan|Ngu_van|Ngoai_ngu|Vat_ly|Hoa_hoc|Sinh_hoc|Lich_su|Dia_ly|GDCD|
+--------+----+-------+---------+------+-------+--------+-------+------+----+
|64002156|   7|   5.25|      3.4|  7.25|      6|    5.25|   null|  null|null|
|64002157| 5.4|   6.25|      1.8|  null|   null|    null|      4|  null|   8|
|64002158| 4.8|      7|        4|  null|   null|    null|   5.25|  null|   8|
|64002159|   6|   6.75|      3.2|  null|   null|    null|   3.25|  null|8.75|
|64002160| 5.2|      7|      2.8|  null|   null|    null|   

### Thay đổi kiểu dữ liệu các trường dữ liệu về đúng dạng của nó

In [4]:
import pyspark.sql.functions as F
import pyspark.sql.functions as f
from pyspark.sql.types import FloatType, StringType, IntegerType

In [5]:
data_all = data_all.withColumn('Toan', F.col('Toan').cast(FloatType()))
data_all = data_all.withColumn('Ngu_van', F.col('Ngu_van').cast(FloatType()))
data_all = data_all.withColumn('Ngoai_ngu', F.col('Ngoai_ngu').cast(FloatType()))
data_all = data_all.withColumn('Vat_ly', F.col('Vat_ly').cast(FloatType()))
data_all = data_all.withColumn('Hoa_hoc', F.col('Hoa_hoc').cast(FloatType()))
data_all = data_all.withColumn('Sinh_hoc', F.col('Sinh_hoc').cast(FloatType()))
data_all = data_all.withColumn('Lich_su', F.col('Lich_su').cast(FloatType()))
data_all = data_all.withColumn('Dia_ly', F.col('Dia_ly').cast(FloatType()))
data_all = data_all.withColumn('GDCD', F.col('GDCD').cast(FloatType()))

data_all.printSchema()

root
 |-- SBD: string (nullable = true)
 |-- Toan: string (nullable = true)
 |-- Ngu_van: string (nullable = true)
 |-- Ngoai_ngu: string (nullable = true)
 |-- Vat_ly: string (nullable = true)
 |-- Hoa_hoc: string (nullable = true)
 |-- Sinh_hoc: string (nullable = true)
 |-- Lich_su: string (nullable = true)
 |-- Dia_ly: string (nullable = true)
 |-- GDCD: string (nullable = true)



In [6]:
data_all = data_all.withColumn('Ma_cum', f.substring(f.col('SBD'), 0, 2))
data_all = data_all.drop_duplicates()

In [7]:
print(data_all.count())

811448


In [8]:
from pyspark.sql.window import Window

data_all = data_all.withColumn('row_num', f.row_number().over(
    Window.partitionBy('SBD').orderBy('SBD')
)).filter("row_num == 1").drop("row_num")

print(data_all.count())

811448


In [9]:
data_all.show()

+--------+----+-------+---------+------+-------+--------+-------+------+----+------+
|     SBD|Toan|Ngu_van|Ngoai_ngu|Vat_ly|Hoa_hoc|Sinh_hoc|Lich_su|Dia_ly|GDCD|Ma_cum|
+--------+----+-------+---------+------+-------+--------+-------+------+----+------+
|01000061| 6.6|    6.5|      4.0|  5.25|    3.5|     2.0|   null|  null|null|    01|
|01000256| 7.2|   7.25|      5.0|  6.75|    6.0|    5.25|   null|  null|null|    01|
|01000622| 9.0|    8.0|      9.0|  null|   null|    null|   5.75|  7.75|8.75|    01|
|01002152| 8.6|    7.5|      8.8|  null|   null|    null|   8.25|   8.0|9.25|    01|
|01002352| 7.2|    6.5|      7.4|  null|   null|    null|   6.25|  6.25|7.75|    01|
|01002463| 7.2|   7.75|     null|  null|   null|    null|   5.25|  7.25| 9.0|    01|
|01002478| 7.4|    7.5|      6.0|  3.75|   7.25|     7.0|   null|  null|null|    01|
|01002876| 7.8|    7.5|      6.2|  null|   null|    null|   2.25|   7.5|7.75|    01|
|01002929| 8.6|    8.5|      9.4|  null|   null|    null|   6.25|

In [10]:
data_all = data_all.orderBy("Ma_cum", "SBD")
data_all.repartition(1).write.format("com.databricks.spark.csv").option("header", "true").save("mydata.csv")

In [11]:
sparkSession.stop()